In [1]:
### Based on https://analyticsindiamag.com/step-by-step-guide-to-implement-multi-class-classification-with-bert-tensorflow/
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)

tensorflow version :  1.15.0
tensorflow_hub version :  0.7.0


In [2]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [3]:
OUTPUT_DIR ='models/23Jan/'
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: models/23Jan/ *****


In [4]:
!nvidia-smi

Thu Jan 23 04:15:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 430.40       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 21%   33C    P8     9W / 250W |  11072MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 21%   25C    P8     8W / 250W |   8618MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
device_to_use = gpus[4]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.experimental.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU')]


In [6]:
session_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True,visible_device_list = '4'))

In [7]:
os.environ["TFHUB_CACHE_DIR"] = '/space/engineering/tfhub_modules'

In [8]:
df = pd.read_csv('data/columbia.csv',sep=',',encoding='utf-8')

In [9]:
df = df.reset_index(drop=True)

In [10]:
df.head(5)

,Session_id,Free_Text,RU_Tag
0,INT-sv1appis11-1539193816745-341861,The the Downpour II jacket being phased out or...,Shopping_Query
1,INT-sv1appis11-1539200026744-344366,hi this is chris ## chris who works here lol #...,None_None
2,INT-sv1appis11-1539201066821-344791,the Canuk Titanium omin-heat 3D outdry Ex Boot?,Shopping_Query
3,INT-sv1appis11-1539201218354-344847,Can you cal me XXXX-XXXX-74 ## Can you call me...,None_None
4,INT-sv1appis11-1539202397866-345288,Im looking for a light weight jacket for clim...,Shopping_Query


In [11]:
LABEL_COLUMN = 'RU_Tag'
DATA_COLUMN = 'Free_Text'

In [12]:
le = LabelEncoder()
le.fit(df[LABEL_COLUMN].unique())

LabelEncoder()

In [13]:
y = df[LABEL_COLUMN]
y = pd.Series(le.transform(y))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df[DATA_COLUMN],y,train_size=0.8,random_state=2019)

In [15]:
df_train = pd.DataFrame(X_train)
df_train[LABEL_COLUMN] = y_train
df_train#.astype({'ru_tag': 'int64'})
df_test = pd.DataFrame(X_test)
df_test[LABEL_COLUMN] = y_test

In [16]:
labels = sorted(df[LABEL_COLUMN].unique())
#labels_actual = sorted(df['ru_tag'].unique())
label_list = range(len(labels))

In [17]:
len(labels)

65

In [18]:
train_InputExamples = df_train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
val_InputExamples = df_test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


In [19]:
train_InputExamples.iloc[0].text_a

'Do you have big and tall sizes?'

In [20]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  Do you have big and tall sizes?

__________
Row 0 - text_b of training set :  None

__________
Row 0 - label of training set :  42


In [21]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
#BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1"
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session(config = session_config) as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [22]:
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)


__________
Row 0 - text_a of training set :  Do you have big and tall sizes?


In [23]:
#Here is what the tokenised sample of the first training set observation looks like
tokenizer.tokenize('what is this')

['what', 'is', 'this']

In [22]:
# We'll set sequences to be at most 20 tokens long.
MAX_SEQ_LENGTH = 50
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 12172


INFO:tensorflow:Writing example 0 of 12172


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 42 (id = 42)


INFO:tensorflow:label: 42 (id = 42)


In [34]:
#Example on first observation in the training set
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("Segment IDs : ", train_features[0].segment_ids)

Sentence :  Do you have big and tall sizes?
------------------------------
Tokens :  ['do', 'you', 'have', 'big', 'and', 'tall', 'sizes', '?']
------------------------------
Input IDs :  [101, 2079, 2017, 2031, 2502, 1998, 4206, 10826, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Input Masks :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Segment IDs :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [23]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [24]:
#A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [25]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 35
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    session_config = session_config)



In [26]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'models/23Jan/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  allow_growth: true
  visible_device_list: "4"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f14e021ab90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'models/23Jan/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  allow_growth: true
  visible_device_list: "4"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f14e021ab90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [27]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [28]:
num_train_steps

1043

In [29]:
#Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 1043 into models/23Jan/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1043 into models/23Jan/model.ckpt.


INFO:tensorflow:Loss for final step: 0.47913998.


INFO:tensorflow:Loss for final step: 0.47913998.


Training took time  0:05:09.611566


In [30]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/opt/custom/python/anaconda3/envs/tf1.15/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 1043: eval_accuracy = 0.8653088, false_negatives = 0.0, false_positives = 3.0, global_step = 1043, loss = 0.6206377, true_negatives = 0.0, true_positives = 3041.0


INFO:tensorflow:Saving dict for global step 1043: eval_accuracy = 0.8653088, false_negatives = 0.0, false_positives = 3.0, global_step = 1043, loss = 0.6206377, true_negatives = 0.0, true_positives = 3041.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1043: models/23Jan/model.ckpt-1043


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1043: models/23Jan/model.ckpt-1043


{'eval_accuracy': 0.8653088,
 'false_negatives': 0.0,
 'false_positives': 3.0,
 'loss': 0.6206377,
 'true_negatives': 0.0,
 'true_positives': 3041.0,
 'global_step': 1043}

In [31]:
# A method to get predictions
def getPrediction(in_sentences):
  #A list to map the actual labels to the predictions
  labels = sorted(df[LABEL_COLUMN].unique())

  #Transforming the test data into BERT accepted form
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  
  #Creating input features for Test data
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

  #Predicting the classes 
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'],prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]


In [32]:
test_data = pd.read_csv('data/val.csv',sep=',',encoding='utf-8')

In [33]:
test_data.head(5)

,Session_id,Free_Text,RU_Tag
0,SyntData09NOV523,I want to buy a boy's rain jacket,Shopping_Query
1,SyntData52553,oh yes,Yes_Give
2,INT-va1appis15-1540083928550-176076,I see a price of XXX.98 but checkout says its ...,Columbia_Others
3,SyntData53658,Do you ship to Oregon?,Shipping_Where
4,SyntData54286,I have a gift card i want to use @ 29.93 ## ye...,GiftCard_Query


In [34]:
test_sentences = test_data[DATA_COLUMN]

In [35]:
predictions = getPrediction(test_sentences)

INFO:tensorflow:Writing example 0 of 3044


INFO:tensorflow:Writing example 0 of 3044


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from models/23Jan/model.ckpt-1043


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [36]:
for i in range(5):
    print(predictions[i])

("I want to buy a boy's rain jacket ", array([ -8.426598  ,  -8.875916  ,  -8.344848  ,  -8.676337  ,
        -8.843563  ,  -7.4173913 ,  -7.7748833 ,  -8.798188  ,
        -6.824803  ,  -8.318573  ,  -7.7107415 , -10.045474  ,
        -8.527266  ,  -8.469207  ,  -6.955068  ,  -8.657051  ,
        -8.760571  ,  -7.915339  ,  -8.043224  ,  -9.230104  ,
        -8.525097  ,  -8.602346  ,  -9.500961  ,  -9.606273  ,
        -8.007634  ,  -9.539378  ,  -9.119832  ,  -9.485687  ,
        -8.054887  ,  -9.394083  ,  -9.117561  ,  -9.154424  ,
        -8.892512  ,  -9.095255  ,  -8.655472  ,  -9.201636  ,
        -9.055048  ,  -8.286728  ,  -8.55259   ,  -8.72656   ,
        -8.668051  ,  -7.648604  ,  -6.92419   ,  -8.39826   ,
        -8.598908  ,  -8.417799  ,  -8.527708  ,  -8.579185  ,
        -8.375906  ,  -8.538064  ,  -8.447292  , -10.223145  ,
        -8.610137  ,  -8.726426  ,  -0.01595645,  -9.217963  ,
        -8.712474  ,  -6.975457  ,  -8.144048  ,  -7.9879856 ,
        -8.37111

In [37]:
enc_labels = []
act_labels = []
for i in range(len(predictions)):
  enc_labels.append(predictions[i][2])
  act_labels.append(predictions[i][3])

In [38]:
pd.DataFrame(enc_labels, columns = ['RU_Tag']).to_excel('data/result.xlsx', index = False)

In [42]:
#Classifying random sentences
tests = getPrediction(['It says valid from 1/1/20 - 3/1/29',
                       'are you hiring',
                       'are you a robot?',
                       'cook for me',
                       'are you a human?',
                       'Can I marry you?',
                       'I will marry you',
                       'can you get married',
                       'marry me',
                       'i want to marry you',
                       'let\'s get married',
                       'Do you fart?',
                       'do you laugh?',
                       'do you cry?',
                       'do you have any emotion?',
                       'Do you believe in God?',
                       'which God do you believe in?',
                       'do you have faith in God?',
                       'you\'re the best bot!',
                       'Wow!',
                       'awesome!',
                       'cancel dish movie pack',
                       'you rock!',
                       'How can I get the Big Ten Network to watch college football',
                       'it says order not present',
                       'it says it ends today',
                       'will it be available today'
                       ])

INFO:tensorflow:Writing example 0 of 27


INFO:tensorflow:Writing example 0 of 27


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from models/23Jan/model.ckpt-1043


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [43]:
tests

[('It says valid from 1/1/20 - 3/1/29',
  array([-6.854724 , -6.3881845, -7.0446844, -6.7011795, -6.6448917,
         -7.096636 , -5.775078 , -6.8049297, -0.5552032, -5.744276 ,
         -5.332964 , -2.2974882, -5.7406063, -6.6633577, -6.9970818,
         -6.363598 , -7.101962 , -6.617673 , -7.811839 , -7.355669 ,
         -6.230256 , -6.8130875, -5.402692 , -5.1103725, -6.7406797,
         -7.290662 , -6.290531 , -6.8612976, -5.1601057, -5.220106 ,
         -6.3910093, -6.359108 , -6.262042 , -4.385978 , -6.673172 ,
         -5.4117947, -7.0340223, -5.0941668, -6.508065 , -5.5013313,
         -5.3045645, -6.2901306, -2.9872444, -4.5052814, -5.758812 ,
         -6.24566  , -4.251865 , -5.024065 , -3.9980986, -5.7346563,
         -6.80023  , -5.4844856, -6.5512295, -6.4327145, -5.9007206,
         -6.7391777, -2.9439507, -3.810323 , -4.7167664, -6.39499  ,
         -5.821184 , -7.1892233, -4.490123 , -7.2826796, -6.612893 ],
        dtype=float32),
  8,
  'Columbia_Others'),
 ('are you 